In [25]:
import pandas as pd
import math
import numpy as np

class Sampler:
    def __init__(self, input_path, ann_file_path):
        '''
        The Sampler class contains sampling methods for the video input data.
        :param input_path: file path for the input videos
        :param ann_file_path: file path for annotation file belonging to the video input data
        '''
        self.input = input_path
        self.ann_file = ann_file_path

    
    def stratifiedSampling(self, trim_len=10, n_fall_samples=3, samples_per_min=1, output_path="/out"):
        '''
        Stratified sampling only samples from videos which contain all three 
        actions (ADL, falling, lying). From these videos, a base rate of fall 
        samples will be sampled uniformly on the fall interval (time where fall happens). 
        ADL and lying activities are sampled based on their ratio of duration based 
        on sample rate.
        :param trim_len: sample video length in seconds (defaults to 10) 
        :param fall_samples: amount of fall samples to collect from videos where 
        falls occur (defaults to 3) 
        :param samples_per_min: amount of samples per minute from ADL and lying activities 
        (defaults to 1)
        :param output_path: filepath for sample outputs
        '''
        
        # Read annotation file to dataframe
        df = pd.read_csv(self.ann_file)
        
        # Filter for only fall videos
        df = df[df['category'] == "Fall"] 
        
        # Reset index after filtering
        df = df.reset_index()
        
        # TODO: Remove
        sample_info={}

        for i in df.index:
            
            # Get timestamps
            fall_start = float(df.loc[i, 'fall_start'])
            fall_end = float(df.loc[i, 'fall_end'])
            lying_start = float(df.loc[i, 'lying_start'])
            lying_end = float(df.loc[i, 'lying_end'])
            video_end = float(df.loc[i, 'length'])
            
            # Calculate action durations
            ADL1_time = fall_start
            ADL2_time = video_end - lying_end
            ADL_time = ADL1_time + ADL2_time
            fall_time = fall_end - fall_start
            lying_time = lying_end - lying_start
            
            # Calculate number of samples for ADL and lying activities
            n_samples = round((ADL_time + lying_time) / 60) * samples_per_min
            n_ADL_samples = round(n_samples * (ADL_time / (ADL_time + lying_time)))
            n_lying_samples = n_samples - n_ADL_samples
            n_ADL1_samples = round((ADL1_time/ADL_time)*n_ADL_samples)
            n_ADL2_samples = n_ADL_samples - n_ADL1_samples
            
            # Sample uniformly on the intervals
            ADL1_samples = np.random.uniform(0, ADL1_time, n_ADL1_samples)
            ADL2_samples = np.random.uniform(lying_end, video_end, n_ADL2_samples)
            fall_samples = np.random.uniform(fall_start, fall_end, n_fall_samples)
            lying_samples = np.random.uniform(lying_start, lying_end, n_lying_samples)
            
            print(f'Video path: {df.loc[i, "video_path"]}')
            print("ADL1_samples: ", ADL1_samples)
            print("ADL2_samples: ", ADL2_samples)
            print("fall_samples: ", fall_samples)
            print("lying_samples: ", lying_samples)
            print()
    


In [27]:
strat = Sampler("nothingfornow", "../data/fall_simulation_data/annotations.csv")

strat.stratifiedSampling(samples_per_min=3)

Video path: data/Fall_Simulation_Data/videos/Fall30_Cam3.avi
ADL1_samples:  [7.70762858 8.29947572]
ADL2_samples:  [96.23741577]
fall_samples:  [26.77860852 25.12724062 25.08253061]
lying_samples:  [48.56114932 50.93237031 76.22962764]

Video path: data/Fall_Simulation_Data/videos/Fall19_Cam4.avi
ADL1_samples:  [ 5.85334979  2.51960384 12.06864939]
ADL2_samples:  [229.63493697]
fall_samples:  [55.17302411 55.53212455 57.73908142]
lying_samples:  [168.02749579 218.65328576 177.75685     80.03421712 217.00161039
 133.62708995 131.02889226 220.84692219]

Video path: data/Fall_Simulation_Data/videos/Fall18_Cam4.avi
ADL1_samples:  [ 2.37071882 87.36245566 76.0052116  57.83063837]
ADL2_samples:  [250.99030674]
fall_samples:  [90.44541367 89.00909486 88.86176711]
lying_samples:  [223.35869143 151.76379406 171.73561923 155.44485236 103.79030908
 132.61813697 145.50658415]

Video path: data/Fall_Simulation_Data/videos/Fall31_Cam3.avi
ADL1_samples:  [22.72786033 47.60331234 60.15377944]
ADL2_sam

# Saved print-outs
print('ADL1 time: ', ADL1_time)
print('ADL2 time: ', ADL2_time)
print('Total ADL time: ', ADL_time)
print('Fall time: ', fall_time)
print('Lying time: ', lying_time)

print("n samples: ", n_samples)
print("n ADL samples: ", n_ADL_samples)
print("n lying samples: ", n_lying_samples)
print("n ADL1 samples: ", n_ADL1_samples)
print("n ADL2 samples: ", n_ADL2_samples)

print("ADL1_samples: ", ADL1_samples)
print("ADL2_samples: ", ADL2_samples)
print("fall_samples: ", fall_samples)
print("lying_samples: ", lying_samples)

In [16]:
np.random.uniform(0,0.5,2)

array([0.29149897, 0.39570417])